In [ ]:
!pip install spacy==3.3.1
# !python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 53.0 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.9
    Uninstalling thinc-8.1.9:
      Successfully uninstalled thinc-8.1.9
  Attempting uninstall: spacy
    Found existing 

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 82 (delta 16), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (82/82), 5.33 MiB | 7.07 MiB/s, done.


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
!nvidia-smi

Sat Apr 29 01:57:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

MessageError: ignored

In [ ]:
cv_data=json.load(open('/content/test2 (5).json','r'))

In [ ]:
len(cv_data)

451

In [ ]:
!pip install spacy-transformers==1.1.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 81.5 MB/s eta 0:00:00


In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

2023-04-29 01:57:55.395718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
nlp = spacy.blank('en')

In [ ]:
def get_spacy_doc(file,data):
    nlp = spacy.blank('en')
    db=DocBin()

    for text,annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot=annot['entities']

        ents=[]
        entity_indices=[]

        for start,end,label in annot:
            skip_entity=False
            for idx in range(start,end):
                if idx in entity_indices:
                    skip_entity=True
                    break
            if skip_entity==True:
                continue

            entity_indices=entity_indices+list(range(start,end))

            try:
                span=doc.char_span(start,end,label=label,alignment_mode='contract')
            except:
                continue

            if span is None:
                err_data=str([start,end])+"   "+str(text)+"\n"
                file.write(err_data)

            else:
                ents.append(span)

        try:
            doc.ents=ents
            db.add(doc)
        except:
            pass
    return db

In [ ]:
import re


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [ ]:
# len(train),len(test)
# from sklearn.model_selection import train_test_split
# train,test = train_test_split(cv,test_size=0.3)

from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size=0.3)

In [ ]:
import spacy

In [ ]:
file =open('error.txt','w')

db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

  1%|          | 2/315 [00:00<00:05, 53.87it/s]


ValueError: ignored

In [ ]:
# skweak-0.3.3
!pip install skweak
import skweak

In [ ]:
new_docs = []
for doc in skweak.utils.docbin_reader("/content/train_data.spacy",spacy_model_name='en_core_web_sm'):
    new_docs.append(doc)

In [ ]:
doc_examples = []
for i,j in enumerate(new_docs):
    spans_ex = [(ent.start_char,ent.end_char,ent.label_) for ent in new_docs[i].ents]
    doc_examples.append([new_docs[i].text,{"entities":spans_ex}])

In [ ]:
doc_examples = trim_entity_spans(doc_examples)


In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(doc_examples,test_size=0.3)

In [ ]:
train

In [ ]:
file =open('error.txt','w')

db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

# Model1: ROBERTA

In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --gpu-id 0 --output ./output --paths.train /content/train_data.spacy --paths.dev /content/test_data.spacy


In [ ]:
# Saving the model
nlp.to_disk('/content/drive/MyDrive/1. Resume_Parser/model-last')

In [ ]:
#Loading Model
roberta_model = spacy.load('/content/drive/MyDrive/1. Resume_Parser/model-last')

# Model2: DistilBERT

In [ ]:
from transformers import DistilBertConfig, DistilBertModel

# Initializing a DistilBERT configuration
configuration = DistilBertConfig()

# Initializing a model (with random weights) from the configuration
model = DistilBertModel(configuration)

# Accessing the model configuration
configuration = model.config

#Model3: GPT2

In [ ]:
from transformers import GPT2Config, GPT2Model

# Initializing a GPT2 configuration
configuration = GPT2Config()

# Initializing a model (with random weights) from the configuration
model = GPT2Model(configuration)

# Accessing the model configuration
configuration = model.config

# Model4: BERT

In [ ]:
from transformers import BertConfig, BertModel

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install spacy==3.3.1
# !pip install thinc==8.0.17

In [ ]:
# !python -m spacy download en_core_web_lg

In [ ]:
# !pip uninstall thinc
# !pip install torch==1.1.0
# !pip install spacy-pytorch-transformers[cuda100]==0.2.0


In [ ]:
#nlp= spacy.load("en_core_web_lg")#path model best

OSError: ignored

In [ ]:
import spacy
import spacy_transformers
nlp=spacy.load("/content/output/model-best")

In [ ]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
backports.zoneinfo            0.2.1
beautifulsoup4                4.6.3
bleach                        6.0.0
blis  

In [ ]:
doc=nlp('my name is lauda lausun i have 8 years of experience I work in microsoft')

In [ ]:
for ent in doc.ents:
  print(ent.text,"  ->>>>",ent.label_)

In [ ]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 81.0 MB/s eta 0:00:00


In [ ]:
import sys, fitz

In [ ]:
fname = '/content/CV-Parsing-using-Spacy-3/data/test/Smith Resume.pdf'
doc = fitz.open(fname)

In [ ]:
doc

In [ ]:
text = " "
for page in doc:
  text = text +str(page.get_text())

In [ ]:
text = text.strip()

In [ ]:
text

'Michael Smith \nBI / Big Data/ Azure \nManchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f \n \n10+ years of Experience in Designing, Development, Administration, Analysis, \nManagement \ninthe \nBusiness \nIntelligence \nData \nwarehousing, \nClient \nServer \nTechnologies, Web-based Applications, cloud solutions and Databases. \nData warehouse: Data analysis, star/ snow flake schema data modeling and design \nspecific todata warehousing and business intelligence environment. \nDatabase: Experience in database designing, scalability, back-up and recovery, \nwriting andoptimizing SQL code and Stored Procedures, creating functions, views, \ntriggers and indexes.  \nCloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL \nAzure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure \ndata lake analytics(U-SQL). \nBig Data: Worked Azure data lake store/analytics for big data processing and Azure \ndata factoryto schedule U-

In [ ]:
text = ' '.join(text.split())

In [ ]:
text

'Michael Smith BI / Big Data/ Azure Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f 10+ years of Experience in Designing, Development, Administration, Analysis, Management inthe Business Intelligence Data warehousing, Client Server Technologies, Web-based Applications, cloud solutions and Databases. Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific todata warehousing and business intelligence environment. Database: Experience in database designing, scalability, back-up and recovery, writing andoptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL). Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factoryto schedule U-SQL jobs. Designed and developed end to end big 

In [ ]:
doc=nlp(text)
for ent in doc.ents:
  print(ent.text,"  ->>>>",ent.label_)

Manchester   ->>>> Location
indeed.com/r/falicent/140749dace5dc26f   ->>>> Email Address
10+   ->>>> Years of Experience
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
2015   ->>>> Graduation Year
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
Microsoft   ->>>> Companies worked at
The University of   ->>>> College Name
problem solving (Less than 1 year), project lifecycle (Less than 1 year), project manager (Less than 1 year), technical assistance. (Less than 1 year) ADDITIONAL INFORMATION Professional Skills Excellent analytical, problem solving, communic